In [2]:
import cplex
import networkx as nx

In [3]:
graph_names = ["brock200-1.mtx", "brock800-4.mtx", 
               "c-fat200-5.mtx", "c-fat500-10.mtx", "hamming10-2.mtx", "hamming10-4.mtx",
               "keller4.mtx", "keller5.mtx", "MANN-a27.mtx",
               "san200-0-7-1.mtx", "sanr400-0-7.mtx"]

In [4]:
graphs_for_testing = []

for graph in graph_names:

    with open("./graphs/"+graph) as file:
        data = file.readlines()
    data = data[2:]
    
    G = nx.Graph()

    for line in data:
        line = line.strip().split()
        u, v = int(line[0])-1, int(line[1])-1
        G.add_edge(u, v)
        
    graphs_for_testing.append((graph, G))

In [5]:
results = []

for g in graphs_for_testing:
    
    prob = cplex.Cplex()
    prob.set_problem_name("Minimum Vertex Cover")
    
    prob.set_problem_type(cplex.Cplex.problem_type.LP)

    prob.objective.set_sense(prob.objective.sense.minimize)

    n = len(g[1].nodes)

    names = [str(i) for i in range(n)]

    # Objective (linear) weights
    w_obj = [1 for i in range(n)]
    # Lower bounds
    low_bnd = [0 for i in range(n)]
    # Upper bounds
    upr_bnd = [1 for i in range(n)]
    prob.variables.add(names=names, obj=w_obj, lb=low_bnd, ub=upr_bnd)

    # Set variable types
    all_int = [(var, prob.variables.type.integer) for var in names]
    prob.variables.set_types(all_int)

    constraints = []
    for e in g[1].edges:
        constraints.append([[str(e[0]), str(e[1])], [1, 1]])

    # Constraint names
    constraint_names = ["".join(x[0]) for x in constraints]

    # Each edge must have at least one vertex
    rhs = [1] * len(constraints)

    # G -> >=
    constraint_senses = ["G"] * len(constraints)

    # And add the constraints
    prob.linear_constraints.add(names=constraint_names,
                                lin_expr=constraints,
                                senses=constraint_senses,
                                rhs=rhs)

    #Time limit
    prob.parameters.timelimit.set(180*60)

    # Solve the problem
    prob.solve()
    
    print("Solution result is: %s" % prob.solution.get_status_string())
    print(prob.solution.get_values())
    
    results.append((g[0], sum(prob.solution.get_values())))

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               10800
Found incumbent of value 200.000000 after 0.00 sec. (0.23 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 12181 rows and 0 columns.
MIP Presolve modified 2653 coefficients.
Reduced MIP has 2653 rows, 200 columns, and 37218 nonzeros.
Reduced MIP has 200 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.58 sec. (339.50 ticks)
Probing time = 0.01 sec. (0.94 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 2653 rows, 200 columns, and 37218 nonzeros.
Reduced MIP has 200 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.08 sec. (49.58 ticks)
Probing time = 0.01 sec. (1.23 ticks)
Clique table members: 2653.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time =

     10    10      786.2836    37      791.0000      750.5614    30495    5.11%
     32    12      767.0916   261      791.0000      750.5614    31602    5.11%
     34     5      750.6148   794      791.0000      750.5614    20004    5.11%
     35    14      778.0839   104      791.0000      750.5614    32950    5.11%
     41    21      787.8129    28      791.0000      750.5614    35348    5.11%
     72    38      787.2636    35      791.0000      750.5949    46549    5.11%
Elapsed time = 213.94 sec. (118714.53 ticks, tree = 0.45 MB, solutions = 4)
    124    49      780.4833    90      791.0000      750.5949    51026    5.11%
    183   111      788.5000    37      791.0000      750.5949    58454    5.11%
    219   114        cutoff            791.0000      750.6148    72075    5.11%
    288   163      781.6486    84      791.0000      750.6148    87023    5.11%
    365   226      765.8856   292      791.0000      750.6148   100011    5.11%
*   366+  161                          790.0

   4571  2492      783.6974    62      790.0000      751.1877   456272    4.91%
   4671  2508      786.8265    44      790.0000      751.1877   458549    4.91%
   4726  2519      781.7974    74      790.0000      751.1877   457439    4.91%
   4828  2675      783.6800    64      790.0000      751.1998   466677    4.91%
   4986  2819      781.0593    78      790.0000      751.1998   475056    4.91%
Elapsed time = 1162.91 sec. (451754.30 ticks, tree = 181.92 MB, solutions = 5)
   5114  2878      781.3604    73      790.0000      751.1998   478051    4.91%
   5228  2900      788.3333    32      790.0000      751.1998   477214    4.91%
   5366  2917      783.6742    59      790.0000      751.1998   480593    4.91%
   5557  3055      785.5859    50      790.0000      751.1998   485719    4.91%
   5642  3071      784.4249    56      790.0000      751.1998   487311    4.91%
   5863  3326        cutoff            790.0000      751.1998   501951    4.91%
   5986  3365        cutoff            79

  23644 15162      769.2249   228      790.0000      751.2263  2213108    4.91%
Elapsed time = 2145.04 sec. (763737.64 ticks, tree = 912.32 MB, solutions = 5)
  23754 15266      776.5719   133      790.0000      751.2263  2222445    4.91%
  23803 15321      780.9979    97      790.0000      751.2263  2230378    4.91%
  23887 15360      768.5293   239      790.0000      751.2263  2269980    4.91%
  23981 15454      775.7232   145      790.0000      751.2263  2278752    4.91%
  24120 15427      770.1371   218      790.0000      751.2263  2303061    4.91%
  24239 15546      779.5752   103      790.0000      751.2263  2312869    4.91%
  24310 15410      752.7487   681      790.0000      751.2263  2303070    4.91%
  24365 15418      752.9936   673      790.0000      751.2263  2306391    4.91%
  24500 15646      782.0882    89      790.0000      751.2263  2345893    4.91%
  24581 15736      768.9216   225      790.0000      751.2263  2384031    4.91%
Elapsed time = 2274.18 sec. (802050.32 ti

  40775 31081      771.5929   195      790.0000      752.7339  4640773    4.72%
  40960 31243      770.3940   207      790.0000      752.7339  4656104    4.72%
  41302 31504      776.0678   134      790.0000      752.8528  4680871    4.70%
  41675 31938      778.2790   115      790.0000      752.8953  4726118    4.70%
  41961 32017      768.2231   245      790.0000      752.9738  4732266    4.69%
  42415 32137      776.1598   140      790.0000      752.9936  4746921    4.68%
  42694 33111      772.5000   188      790.0000      753.0188  4824872    4.68%
  43234 33258      768.4432   239      790.0000      753.1216  4835769    4.67%
Elapsed time = 3619.32 sec. (1198721.80 ticks, tree = 1568.59 MB, solutions = 5)
  43484 33761      775.7072   139      790.0000      753.2007  4885132    4.66%
  43868 33863      770.8687   200      790.0000      753.2348  4896871    4.65%
  44328 34699      774.0725   161      790.0000      753.2655  4997303    4.65%
  44628 34631      768.7044   228      

...

Reduced MIP has 171 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.06 sec. (37.46 ticks)
Probing time = 0.01 sec. (0.74 ticks)
Clique table members: 3497.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.24 sec. (153.06 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                          171.0000        0.0000           100.00%
*     0+    0                          162.0000        0.0000           100.00%
*     0+    0                          156.0000        0.0000           100.00%
      0     0      147.5426   143      156.0000      147.5426       30    5.42%
      0     0      150.1960   150      156.0000     Cuts: 185     1328    3.72%
      0     0      150.9444   150      156.0000      Cuts: 70     1767    3.24%
      0   

     90    12      738.2375   643      745.0000      737.9858   179519    0.94%
     94    14      738.2807   646      745.0000      737.9858   180552    0.94%
     96    12      738.3236   644      745.0000      738.0683   199178    0.93%
    100    13      738.3500   642      745.0000      738.0683   200453    0.93%
    103    15      738.4029   650      745.0000      738.0683   202454    0.93%
    106    16      738.4397   642      745.0000      738.0683   203809    0.93%
    107    11      738.4772   639      745.0000      738.1714   225180    0.92%
Elapsed time = 239.90 sec. (106165.85 ticks, tree = 0.03 MB, solutions = 3)
    112    13      738.5361   638      745.0000      738.1714   227353    0.92%
    116    14      738.5635   643      745.0000      738.1714   228339    0.92%
    118     7        cutoff            745.0000      738.3027   244691    0.90%
    122    11      738.6914   632      745.0000      738.3027   245344    0.90%
    126    13      738.7488   635      745.0

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               10800
Found incumbent of value 378.000000 after 0.00 sec. (1.09 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 70317 rows and 0 columns.
MIP Presolve modified 234 coefficients.
Reduced MIP has 234 rows, 378 columns, and 21083 nonzeros.
Reduced MIP has 378 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.41 sec. (302.92 ticks)
Probing time = 0.02 sec. (2.52 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 234 rows, 378 columns, and 21083 nonzeros.
Reduced MIP has 378 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.04 sec. (22.64 ticks)
Probing time = 0.02 sec. (2.65 ticks)
Clique table members: 234.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.0

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               10800
Found incumbent of value 400.000000 after 0.00 sec. (0.87 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 53624 rows and 0 columns.
MIP Presolve modified 2245 coefficients.
Reduced MIP has 2245 rows, 400 columns, and 31449 nonzeros.
Reduced MIP has 400 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.42 sec. (214.56 ticks)
Probing time = 0.02 sec. (1.36 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 2245 rows, 400 columns, and 31449 nonzeros.
Reduced MIP has 400 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.06 sec. (37.85 ticks)
Probing time = 0.01 sec. (1.81 ticks)
Clique table members: 2245.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time =

   8958  6016      390.2553    42      392.0000      377.6304   646447    3.67%
   9371  6213      387.2091    68      392.0000      377.7645   669540    3.63%
   9779  6373      386.7634    79      392.0000      378.0929   679245    3.55%
  10071  6672      390.1373    30      392.0000      378.4487   702490    3.46%
  10488  7110      389.7915    43      392.0000      378.6663   732773    3.40%
  10928  7260      390.0759    42      392.0000      378.9907   748740    3.32%
  11361  7782      389.4737    47      392.0000      379.4109   783196    3.21%
Elapsed time = 168.41 sec. (76008.56 ticks, tree = 76.33 MB, solutions = 4)
  11767  7894      388.6218    52      392.0000      379.6639   792273    3.15%
  12170  8218      388.1341    62      392.0000      380.1464   819321    3.02%
  12660  8547      388.6062    57      392.0000      380.5943   845688    2.91%
  13125  8918      390.6022    38      392.0000      381.0210   868103    2.80%
  13513  9076      389.6621    47      392.0

In [13]:
results

[('brock200-1.mtx', 194.0),
 ('brock800-4.mtx', 790.0),
 ('c-fat200-5.mtx', 197.0),
 ('c-fat500-10.mtx', 496.0),
 ('hamming10-2.mtx', 1022.0),
 ('hamming10-4.mtx', 1004.0),
 ('keller4.mtx', 156.0),
 ('keller5.mtx', 745.0),
 ('MANN-a27.mtx', 375.0),
 ('san200-0-7-1.mtx', 191.0),
 ('sanr400-0-7.mtx', 392.0)]